In [13]:


from functools import partial

import numpy as np
import numba

from bokeh.plotting import figure
from bokeh.io import output_notebook, show

import datashader as ds
import datashader.transfer_functions as tf
from datashader.bokeh_ext import InteractiveImage

import xarray as xr

from distributed import Client,Executor
from dask import delayed
from PIL import Image


In [3]:
output_notebook()

Loading BokehJS ...

In [4]:
xmin=-2.5
xmax=1.0
ymin=-1.25
ymax=1.25

maxiterations=np.uint64(5000)

In [5]:
client = Client(n_workers=8, threads_per_worker=2, memory_limit='2GB')
client
e=Executor(client.cluster)
e



/opt/conda/lib/python3.9/site-packages/distributed/node.py:180: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 43851 instead
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/distributed/client.py:4785: UserWarning: Executor has been renamed to Client
  warnings.warn("Executor has been renamed to Client")


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:43851/status,
Dashboard: http://127.0.0.1:43851/status,Workers: 8
Total threads: 16,Total memory: 14.90 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33613,Workers: 8
Dashboard: http://127.0.0.1:43851/status,Total threads: 16
Started: Just now,Total memory: 14.90 GiB
Comm: tcp://127.0.0.1:38845,Total threads: 2
Dashboard: http://127.0.0.1:44545/status,Memory: 1.86 GiB
Nanny: tcp://127.0.0.1:41239,


In [6]:
#@numba.jit("uint64(complex128,uint64)", nopython=True, nogil=True)
@delayed
def mandel(c, maxiterations):
    z=c
    for iteration in range(maxiterations):
        if abs(z)>2.0:
            return iteration
        z=z**2+c
    return maxiterations

In [10]:
mandel_p=partial(mandel.compute(), maxiterations=maxiterations)

def chunked_mandel(chunk):
    return [mandel_p(c) for c in chunk]

In [11]:
def update_image(x_range, y_range, w, h):
    xmin, xmax=x_range
    ymin, ymax=y_range
    
    x=np.linspace(xmin, xmax, w, dtype=np.float64)
    y=np.linspace(ymin, ymax, h, dtype=np.float64)
    c=x+y[:, None]*1j
    cr=c.ravel()
    cc=np.array_split(cr, h)
    
    futures=e.map(chunked_mandel, cc)
    dresults=e.gather(iter(futures))
    iterations=list(dresults)
    
    image=tf.shade(xr.DataArray(np.vstack(iterations)),
                        cmap=["yellow","red", "black"],
                        how="log")
    return image

In [21]:
%%time
tools="pan,wheel_zoom,box_zoom,reset"

p = figure(tools=tools, plot_width=980, plot_height=600,
        x_range=(xmin, xmax), y_range=(ymin, ymax),
        outline_line_color=None, background_fill_color='blue',
        min_border=0, min_border_left=0, min_border_right=0, min_border_top=0, min_border_bottom=0)

p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

x_range=(xmin, xmax)
y_range=(ymin, ymax)

a=update_image(x_range,y_range,980,600)
print(type(a.as_numpy))
#InteractiveImage(p, update_image)


KeyboardInterrupt: 

In [26]:
#print(type(a.to_numpy()))
name='mandelbrotv2.jpg'
Image.fromarray(a.to_numpy(), mode='HSV').convert('RGB').save(name)    

<class 'numpy.ndarray'>
